In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110320

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,145607.0422
2018-02-28,67087.6232
2018-03-31,131647.7219
2018-04-30,168672.7025
2018-05-31,70776.5319
2018-06-30,94274.7313
2018-07-31,98552.1124
2018-08-31,117720.3484
2018-09-30,125467.3861


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    145607.0422
 2018-02-28     67087.6232
 2018-03-31    131647.7219
 2018-04-30    168672.7025
 2018-05-31     70776.5319
 2018-06-30     94274.7313
 2018-07-31     98552.1124
 2018-08-31    117720.3484
 2018-09-30    125467.3861
 2018-10-31    130817.4812
 2018-11-30    174002.2767
 2018-12-31    215371.0814
 2019-01-31    296103.5606
 2019-02-28    142491.9362
 2019-03-31    341726.1065
 2019-04-30    159671.4568
 2019-05-31    110716.8973
 2019-06-30    179242.5742
 2019-07-31    149740.6729
 2019-08-31    111039.9092
 2019-09-30    117303.4801
 2019-10-31    111213.7562
 2019-11-30    164046.1295
 2019-12-31    319323.1490
 2020-01-31    186538.8423
 2020-02-29    203205.7531
 2020-03-31    214997.5599
 2020-04-30    265114.7509
 2020-05-31    119556.8643
 2020-06-30    141781.6717
 2020-07-31    117495.0729
 2020-08-31     88898.7561
 2020-09-30     93199.3533
 2020-10-31    111158.0042
 2020-11-30    154756.4378
 2020-12-31    162502.9153

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -1.864370
p-value : 0.349003
Critical Values :
	1%: -3.646135
	5%: -2.954127
	10%: -2.615968


In [8]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -4.967290
p-value : 0.000026
Critical Values :
	1%: -3.646135
	5%: -2.954127
	10%: -2.615968


In [9]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1050.685, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1050.788, Time=0.10 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1050.830, Time=0.04 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1050.816, Time=0.06 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1047.044, Time=0.19 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1048.615, Time=0.25 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1049.610, Time=0.08 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1050.769, Time=0.05 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1051.601, Time=0.08 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1052.330, Time=0.10 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1047.954, Time=0.21 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1049.628, Time=0.04 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1051.618, Time=0.05 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1054.394, Time=0.18 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=1053.847, Time=0.23 sec
 ARIMA(3,0

ARIMA(order=(0, 0, 4), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [10]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(0,0,4))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(0, 4)   Log Likelihood                -517.412
Method:                       css-mle   S.D. of innovations          50890.605
Date:                Sat, 11 Sep 2021   AIC                           1046.825
Time:                        03:57:22   BIC                           1057.251
Sample:                             0   HQIC                          1050.647
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.401e+05   1.76e+04      7.977      0.000    1.06e+05    1.74e+05
ma.L1.y        0.0169      0.160      0.105      0.916      -0.297       0.331
ma.L2.y        0.1992      0.119      1.678      0.0

# 예측하기

In [12]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([106253.94788237, 140057.00378604]), array([50890.60531477, 50897.84657155]), array([[  6510.19431399, 205997.70145076],
       [ 40299.05761516, 239814.94995693]]))
